In [1]:
import sqlite3 as dbms

conn = dbms.connect("students.sqlite3")
cursor = conn.cursor()

In [2]:
with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f:
    drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f:
    create_ddl = f.read()

In [3]:
if True:
    cursor.executescript(drop_ddl)
    conn.commit()
if True:
    cursor.executescript(create_ddl)
    conn.commit()

In [4]:
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, PrimaryKeyConstraint, UniqueConstraint, ForeignKeyConstraint
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

In [5]:
engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
null = True
class Program(DeclBase):
    __tablename__ = 'programs'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    program_courses = relationship("Programs_course", backref="program")

    def __init__(self, name):
        self.name = name

class Student(DeclBase):
    __tablename__ = 'students'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program
        
class Course(DeclBase):
    __tablename__ = 'courses'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    programs_courses = relationship("Programs_course", backref="course")
    marks = relationship("Mark", backref="course")
    
    def __init__(self, name):
        self.name = name 
        
class Programs_course(DeclBase):
    __tablename__ = 'programs_courses'
    __table_args__ = {'extend_existing': True}
    __table_args__ = (PrimaryKeyConstraint('semester_number', 'course_id', 'program_id'),)
   
    semester_number = Column(Integer)
    course_id = Column(Integer, ForeignKey('courses.id'))
    program_id = Column(Integer, ForeignKey('programs.id'))
    
    def __init__(self, semester_number, course, program):
        self.semester_number = semester_number
        self.course = course
        self.program = program
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    __table_args__ = {'extend_existing': True}
    __table_args__ = (PrimaryKeyConstraint('student_id', 'course_id'),)
    mark = Column(Integer)
    
    student_id = Column(Integer, ForeignKey('students.id'))
    course_id = Column(Integer, ForeignKey('courses.id'))
    
    def __init__(self, mark, student, course):
        self.mark = mark
        self.student = student
        self.course = course

In [7]:
print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)

Программы и студенты


In [8]:
se1 = Program("Программная инженерия")
se2 = Program("Теоретическая физика")

st1 = Student("002002", "Михайлов", "Артём", "Ильич", se1)
st2 = Student("002003", "Простакова", "Виолетта", "Игоревна", se1)
st3 = Student("002004", "Крючков", "Ростислав", "Макарович", se2)
st4 = Student("002005", "Лосева", "Дарья", "Никитична", se2)

session.add_all([st1, st2, st3, st4])
session.commit()

In [9]:
co1 = Course("Нанобиотехнологии")
se3 = Program("Биоинформатика")

pc1 = Programs_course("2", co1, se3)

session.add(pc1)
try:
    session.commit()
except:
    raise
finally:
    session.close()

In [10]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name, s.surname, s.patronymic, s.card)
    for pc in p.program_courses:
        print("   - ", pc.program.name, pc.course.name, pc.semester_number)
session.commit()

Программы и студенты
Программа:  Программная инженерия
-  Артём Михайлов Ильич 002002
-  Виолетта Простакова Игоревна 002003
Программа:  Теоретическая физика
-  Ростислав Крючков Макарович 002004
-  Дарья Лосева Никитична 002005
Программа:  Биоинформатика
   -  Биоинформатика Нанобиотехнологии 2


In [11]:
print("Курсы: ")
for p in session.query(Course):
    print("Курс: ", p.name)

Курсы: 
Курс:  Нанобиотехнологии


In [12]:
pr = session.query(Program).filter_by(name="Биоинформатика")[0]
st = session.query(Student).filter_by(name="Виолетта")[0]

st.program = pr
session.commit()

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)
session.commit()

Программы и студенты
Программа:  Программная инженерия
-  Артём
Программа:  Теоретическая физика
-  Ростислав
-  Дарья
Программа:  Биоинформатика
-  Виолетта


In [13]:
co2 = Course("Нанотехнологии")

pc2 = Programs_course("4", co2, se2)

session.add(pc2)
try:
    session.commit()
except:
    raise

In [14]:
print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name, s.surname, s.patronymic, s.card)
    for pc in p.program_courses:
        print("   - ", pc.program.name, pc.course.name, pc.semester_number)
session.commit()

Программы и студенты
Программа:  Программная инженерия
-  Артём Михайлов Ильич 002002
Программа:  Теоретическая физика
-  Ростислав Крючков Макарович 002004
-  Дарья Лосева Никитична 002005
   -  Теоретическая физика Нанотехнологии 4
Программа:  Биоинформатика
-  Виолетта Простакова Игоревна 002003
   -  Биоинформатика Нанобиотехнологии 2


In [15]:
print("Курсы")
for p in session.query(Course):
    print("Курс: ", p.name, p.id)

Курсы
Курс:  Нанобиотехнологии 1
Курс:  Нанотехнологии 2


In [16]:
mr1 = Mark("4", st3, co2)

session.add(mr1)
try:
    session.commit()
except:
    raise
finally:
    session.close()

In [17]:
print("Студенты и оценки: ")
for p in session.query(Student):
    print("Студент: ", p.name)
    for pc in p.marks:
        print("- ", pc.student.name, pc.course.name, pc.mark)
session.commit()

Студенты и оценки: 
Студент:  Артём
Студент:  Виолетта
Студент:  Ростислав
-  Ростислав Нанотехнологии 4
Студент:  Дарья


In [18]:
print("Курсы")
for p in session.query(Course):
    print("Курс: ", p.name, p.id)

Курсы
Курс:  Нанобиотехнологии 1
Курс:  Нанотехнологии 2
